In [23]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
from spotipy.oauth2 import SpotifyOAuth
import spotipy.util as util
import sys

In [12]:
#client id and secret for my application
client_id = '7121fea2c5374cb2a4915ce25eae38ad'
client_secret= '38eae86b44974aa59bbe404c6135998f'

In [25]:
scope = 'user-library-read'

if len(sys.argv) > 1:
    username = sys.argv[1]
else:
    print("Usage: %s username" % (sys.argv[0],))
    sys.exit()

In [26]:
auth_manager = SpotifyClientCredentials(client_id=client_id, client_secret=client_secret)
sp = spotipy.Spotify(auth_manager=auth_manager)

In [27]:
token = util.prompt_for_user_token(scope, client_id= client_id, client_secret=client_secret, redirect_uri='http://127.0.0.1:5000')

In [28]:
sp = spotipy.Spotify(auth=token)

In [35]:
#gather playlist names and images. 
#images aren't going to be used until I start building a UI
id_name = {}
list_photo = {}
for i in sp.current_user_playlists()['items']:

    id_name[i['name']] = i['uri'].split(':')[2]
    list_photo[i['uri'].split(':')[2]] = i['images'][0]['url']

In [36]:
id_name

{'WILI': '6Bdvs8s6DqK1FsL6R9Xhox', 'Monster': '1KYW4t2n4tAr2IhJc3pNg0'}

In [37]:
def create_necessary_outputs(playlist_name,id_dic, df):
    """ 
    Pull songs from a specific playlist.

    Parameters: 
        playlist_name (str): name of the playlist you'd like to pull from the spotify API
        id_dic (dic): dictionary that maps playlist_name to playlist_id
        df (pandas dataframe): spotify datafram
        
    Returns: 
        playlist: all songs in the playlist THAT ARE AVAILABLE IN THE KAGGLE DATASET
    """
    
    #generate playlist dataframe
    playlist = pd.DataFrame()
    playlist_name = playlist_name

    for ix, i in enumerate(sp.playlist(id_dic[playlist_name])['tracks']['items']):
        #print(i['track']['artists'][0]['name'])
        playlist.loc[ix, 'artist'] = i['track']['artists'][0]['name']
        playlist.loc[ix, 'name'] = i['track']['name']
        playlist.loc[ix, 'id'] = i['track']['id'] # ['uri'].split(':')[2]
        playlist.loc[ix, 'url'] = i['track']['album']['images'][1]['url']
        playlist.loc[ix, 'date_added'] = i['added_at']

    playlist['date_added'] = pd.to_datetime(playlist['date_added'])  
    
    playlist = playlist[playlist['id'].isin(df['id'].values)].sort_values('date_added',ascending = False)
    
    return playlist

In [38]:
import pandas as pd

# create an empty dataframe with the specified columns
df = pd.DataFrame(columns=['artist', 'name', 'id', 'url', 'date_added'])

In [41]:
playlist_EDM = create_necessary_outputs('WILI', id_name,df)
#playlist_chill = create_necessary_outputs('chill',id_name, spotify_df)
#playlist_classical = create_necessary_outputs('Epic Classical',id_name, spotify_df)

In [43]:
playlist_EDM

,artist,name,id,url,date_added


In [17]:
import os 
from requests import post, get
from dotenv import load_dotenv
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
from spotipy.oauth2 import SpotifyOAuth


import csv

load_dotenv()

client_id = os.getenv('7121fea2c5374cb2a4915ce25eae38ad')
client_secret = os.getenv('38eae86b44974aa59bbe404c6135998f')

rap_playlist_id = '6Bdvs8s6DqK1FsL6R9Xhox'

scopes = 'user-read-currently-playing user-top-read'


def get_playlist_tracks(playlist_name, playlist_id): 
    '''
    this function retrieves the users playlist, get all of the tracks in that playlist, then gets the audio features of them,
      finally saves them in csv file
    '''

    # get the number of tracks in the list
    playlist = sp.playlist(playlist_id=playlist_id)
    total_tracks = playlist['tracks']['total']
    print("the total number of tracks : ", total_tracks)
    
    
    # Let's get the all the track ids of our playlist
    i = 0
    playlist_tracks= []
    while total_tracks > 0 :
        offset = i 
        for j in range(len(sp.playlist_items(playlist_id=playlist_id, limit= 100)['items'])) :
            try :
                track_id = sp.playlist_items(playlist_id=playlist_id, offset=offset)['items'][j]['track']['id'] # adds to a list of track ids 
                playlist_tracks.append(track_id)
                print('track ',j, 'has the id : ', track_id)
            except :
                break
        i += 100
        total_tracks -= 100
    
    print("*"*200)
    
    # Let's now get the features of the tracks ids we got from the playlist  
    track_features = []
    for k in range(len(playlist_tracks)): 
        track_id = playlist_tracks[k]
        feature_dic = sp.audio_features(track_id)[0]
        feature_dic.pop('duration_ms') # no need for duration
        track_ft = list(feature_dic.values())
        no_str_fts = [item for item in track_ft if not isinstance(item, str)]
        print("track ", k ,"has this features :", no_str_fts)
        track_features.append(no_str_fts)
    
    # Let's now right this playlist content in a csv file
    print("now let's write "+playlist_name+" in a csv file :")
    with open(playlist_name+'_playlist.csv', "w") as f :
        writer = csv.writer(f)

        for row in track_features : 
            writer.writerow(row)


    return track_features 

sp = spotipy.Spotify(auth_manager=spotipy.SpotifyOAuth(client_id = client_id, client_secret = client_secret , redirect_uri="http://127.0.0.1:5000", scope=scopes, requests_timeout = 10))

try:
    # Let's extract the name and id of my playlists 
    my_playlists = []
    for i in range(10) : 
        playlist_name, playlist_id = sp.current_user_playlists()['items'][i]['name'], sp.current_user_playlists()['items'][i]['id']
        tmp = [playlist_name, playlist_id]
        my_playlists.append(tmp)
        print(playlist_name, playlist_id)

    # now let's write our playlists in csv files 
    for playlist_name , playlist_id in my_playlists : # for 
        playlist_tracks = get_playlist_tracks(playlist_name, playlist_id)

    ''' 
    # my top artists :o
    my_top_artists = []
    for j in range(10) :
        top_artist, artist_id = sp.current_user_top_artists()['items'][j]['name'], sp.current_user_top_artists()['items'][j]['name']
        my_top_artists.append([top_artist, artist_id])
        print(top_artist)'''

except spotipy.SpotifyOauthError as e: # refresh the access token 
    sp = spotipy.Spotify(auth_manager=spotipy.SpotifyOAuth(client_id = client_id, client_secret = client_secret , redirect_uri="http://127.0.0.1:5000", scope=scopes))

SpotifyOauthError: No client_id. Pass it or set a SPOTIPY_CLIENT_ID environment variable.

In [ ]:
#    nex tesrg 

In [5]:
import requests

client_id = '7121fea2c5374cb2a4915ce25eae38ad'
client_secret = '38eae86b44974aa59bbe404c6135998f'

# Step 1: Obtain an access token using the Client Credentials Flow
def get_access_token(client_id, client_secret):
    auth_url = 'https://accounts.spotify.com/api/token'
    auth_data = {
        'grant_type': 'client_credentials'
    }
    auth_response = requests.post(auth_url, data=auth_data, auth=(client_id, client_secret))
    return auth_response.json()['access_token']

access_token = get_access_token(client_id, client_secret)

# Step 2: Fetch the list of playlists
def get_playlists(access_token):
    headers = {
        'Authorization': f'Bearer {access_token}'
    }
    playlists_url = 'https://api.spotify.com/v1/me/playlists'
    playlists_response = requests.get(playlists_url, headers=headers)
    return playlists_response.json()['items']

playlists = get_playlists(access_token)

# Step 3: Fetch the tracks for each playlist
def get_playlist_tracks(access_token, playlist_id):
    headers = {
        'Authorization': f'Bearer {access_token}'
    }
    tracks_url = f'https://api.spotify.com/v1/playlists/{playlist_id}/tracks'
    tracks_response = requests.get(tracks_url, headers=headers)
    return tracks_response.json()['items']

for playlist in playlists:
    print(f"Playlist: {playlist['name']}")
    tracks = get_playlist_tracks(access_token, playlist['id'])
    for track in tracks:
        print(f"  Track: {track['track']['name']} by {', '.join([artist['name'] for artist in track['track']['artists']])}")
    print()




KeyError: 'items'

In [1]:
import requests
import json
import base64
import webbrowser
from urllib.parse import urlencode, urlparse, parse_qs
from http.server import BaseHTTPRequestHandler, HTTPServer

client_id = '7121fea2c5374cb2a4915ce25eae38ad'
client_secret = '38eae86b44974aa59bbe404c6135998f'
redirect_uri = 'http://127.0.0.1:5000'

# Step 1: Obtain an authorization code
auth_url = 'https://accounts.spotify.com/authorize'
auth_params = {
    'client_id': client_id,
    'response_type': 'code',
    'redirect_uri': redirect_uri,
    'scope': 'playlist-read-private playlist-read-collaborative'
}
auth_query = urlencode(auth_params)
webbrowser.open(f'{auth_url}?{auth_query}')

# Step 2: Get the authorization code from the redirected URL
redirected_url = input('Enter the redirected URL: ')
parsed_url = urlparse(redirected_url)
auth_code = parse_qs(parsed_url.query)['code'][0]

# Step 3: Exchange the authorization code for an access token
token_url = 'https://accounts.spotify.com/api/token'
token_data = {
    'grant_type': 'authorization_code',
    'code': auth_code,
    'redirect_uri': redirect_uri
}
client_creds = f'{client_id}:{client_secret}'
client_creds_b64 = base64.b64encode(client_creds.encode()).decode()
token_headers = {
    'Authorization': f'Basic {client_creds_b64}'
}
token_response = requests.post(token_url, data=token_data, headers=token_headers)
access_token = token_response.json()['access_token']

# Step 4: Fetch the list of playlists
def get_playlists(access_token):
    headers = {
        'Authorization': f'Bearer {access_token}'
    }
    playlists_url = 'https://api.spotify.com/v1/me/playlists'
    playlists_response = requests.get(playlists_url, headers=headers)
    return playlists_response.json()['items']

playlists = get_playlists(access_token)

# Step 5: Fetch the tracks for each playlist
def get_playlist_tracks(access_token, playlist_id):
    headers = {
        'Authorization': f'Bearer {access_token}'
    }
    tracks_url = f'https://api.spotify.com/v1/playlists/{playlist_id}/tracks'
    tracks_response = requests.get(tracks_url, headers=headers)
    return tracks_response.json()['items']

for playlist in playlists:
    print(f"Playlist: {playlist['name']}")
    tracks = get_playlist_tracks(access_token, playlist['id'])
    for track in tracks:
        print(f"  Track: {track['track']['name']} by {', '.join([artist['name'] for artist in track['track']['artists']])}")
    print()


In [1]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials

In [2]:
cid = '7121fea2c5374cb2a4915ce25eae38ad'
secret = '38eae86b44974aa59bbe404c6135998f'
#Authentication - without user
client_credentials_manager = SpotifyClientCredentials(client_id=cid, client_secret=secret)
sp = spotipy.Spotify(client_credentials_manager = client_credentials_manager)

In [3]:
playlist_link = "https://open.spotify.com/playlist/37i9dQZEVXbNG2KDcFcKOF?si=1333723a6eff4b7f"
playlist_URI = playlist_link.split("/")[-1].split("?")[0]
track_uris = [x["track"]["uri"] for x in sp.playlist_tracks(playlist_URI)["items"]]

In [4]:
for track in sp.playlist_tracks(playlist_URI)["items"]:
    #URI
    track_uri = track["track"]["uri"]
    
    #Track name
    track_name = track["track"]["name"]
    
    #Main Artist
    artist_uri = track["track"]["artists"][0]["uri"]
    artist_info = sp.artist(artist_uri)
    
    #Name, popularity, genre
    artist_name = track["track"]["artists"][0]["name"]
    artist_pop = artist_info["popularity"]
    artist_genres = artist_info["genres"]
    
    #Album
    album = track["track"]["album"]["name"]
    
    #Popularity of the track
    track_pop = track["track"]["popularity"]

In [5]:
sp.audio_features(track_uri)[0]


{'danceability': 0.429,
 'energy': 0.661,
 'key': 11,
 'loudness': -7.227,
 'mode': 1,
 'speechiness': 0.0281,
 'acousticness': 0.00239,
 'instrumentalness': 0.000121,
 'liveness': 0.234,
 'valence': 0.285,
 'tempo': 173.372,
 'type': 'audio_features',
 'id': '3AJwUDP919kvQ9QcozQPxg',
 'uri': 'spotify:track:3AJwUDP919kvQ9QcozQPxg',
 'track_href': 'https://api.spotify.com/v1/tracks/3AJwUDP919kvQ9QcozQPxg',
 'analysis_url': 'https://api.spotify.com/v1/audio-analysis/3AJwUDP919kvQ9QcozQPxg',
 'duration_ms': 266773,
 'time_signature': 4}

In [9]:
track_uri

'spotify:track:3AJwUDP919kvQ9QcozQPxg'